#### Loading and preparing the PCam data for training deep learning models using tensorflow dataset (tfds)

In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"   # Delete if you have GPU's available
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
 
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from keras import layers, losses
from keras.datasets import mnist
from keras.models import Model

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses

from tensorflow.keras.models import Model

Defining a function that splits images and labels and one-hot-encodes the labels

In [28]:
def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label


In [2]:
def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, image  # Use the image itself as the target label

In [4]:
import tensorflow_datasets as tfds

# Corrected file path with escaped backslashes and a missing comma
ds1, ds2, ds3 = tfds.load('patch_camelyon',
                         split=['train[:20%]', 'test[:5%]', 'validation[:5%]'],
                         data_dir="C:\\Users\\caspe\\OneDrive\\Skrivebord\\AvendtML_Eksamen",
                         download=False,
                         shuffle_files=True)


Next we simple transform the data (by the function convert sample described previously) and getting ready for training by splitting it into batches.

In [5]:
train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)
print("Shape of the training data:", train_dataset)
print("Shape of the validation data:", validation_dataset)
print("Shape of the testing data:", test_dataset)

Shape of the training data: <_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None))>
Shape of the validation data: <_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None))>
Shape of the testing data: <_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None))>


In [7]:
# Definition of the Autoencoder model as a subclass of the TensorFlow Model class

class SimpleAutoencoder(Model):
	def __init__(self,latent_dimensions , data_shape):
		super(SimpleAutoencoder, self).__init__()
		self.latent_dimensions = latent_dimensions
		self.data_shape = data_shape

		# Encoder architecture using a Sequential model
		self.encoder = tf.keras.Sequential([
			layers.Flatten(),
			layers.Dense(latent_dimensions, activation='relu'),
		])

		# Decoder architecture using another Sequential model
		self.decoder = tf.keras.Sequential([
			layers.Dense(tf.math.reduce_prod(data_shape), activation='sigmoid'),
			layers.Reshape(data_shape)
		])

	# Forward pass method defining the encoding and decoding steps
	def call(self, input_data):
		encoded_data = self.encoder(input_data)
		decoded_data = self.decoder(encoded_data)
		return decoded_data

# Extracting shape information from the testing dataset
input_data_shape = next(iter(test_dataset))[0].shape[1:]

# Specifying the dimensionality of the latent space
latent_dimensions = 64

# Creating an instance of the SimpleAutoencoder model
simple_autoencoder = SimpleAutoencoder(latent_dimensions, input_data_shape)






In [13]:
# Model compilation
simple_autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

# Training the model
simple_autoencoder.fit(train_dataset, 
                       epochs=10,
                       validation_data=validation_dataset)

Epoch 1/10
1639/1639 [==============================] - 144s 87ms/step - loss: 0.0407 - val_loss: 0.0433
Epoch 2/10
1639/1639 [==============================] - 131s 80ms/step - loss: 0.0403 - val_loss: 0.0429
Epoch 3/10
1639/1639 [==============================] - 163s 99ms/step - loss: 0.0398 - val_loss: 0.0434
Epoch 4/10
1639/1639 [==============================] - 136s 83ms/step - loss: 0.0395 - val_loss: 0.0421
Epoch 5/10
1639/1639 [==============================] - 122s 74ms/step - loss: 0.0392 - val_loss: 0.0426
Epoch 6/10
1639/1639 [==============================] - 109s 67ms/step - loss: 0.0390 - val_loss: 0.0422
Epoch 7/10
1639/1639 [==============================] - 110s 67ms/step - loss: 0.0388 - val_loss: 0.0418
Epoch 8/10
1639/1639 [==============================] - 108s 66ms/step - loss: 0.0387 - val_loss: 0.0414
Epoch 9/10
1639/1639 [==============================] - 103s 63ms/step - loss: 0.0386 - val_loss: 0.0412
Epoch 10/10
1639/1639 [==============================] 

TypeError: Exception encountered when calling layer 'sequential' (type Sequential).

Inputs to a layer should be tensors. Got '<_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None))>' (of type <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>) as input for layer 'flatten'.

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=<_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None))>
  • training=None
  • mask=None